In [1]:
import pandas as pd
import re

raw_df = pd.read_csv("input.csv")
expected_df = pd.read_csv("expected.csv")

In [2]:
raw_df.columns.to_list()

['holdings_id',
 'instance_created_date',
 'instance_id',
 'instance_primary_contributor',
 'instance_title',
 'instance_updated_date',
 'item_barcode',
 'item_level_call_number',
 'item_level_call_number_typeid',
 'item_level_call_number_type_name',
 'item_copy_number',
 'item_created_date',
 'item_effective_call_number',
 'item_effective_call_number_typeid',
 'item_effective_call_number_type_name',
 'item_effective_library_code',
 'item_effective_library_id',
 'item_effective_library_name',
 'item_effective_location_id',
 'item_effective_location_name',
 'item_hrid',
 'id',
 'item_material_type_id',
 'item_material_type',
 'item_permanent_location_id',
 'item_permanent_location_name',
 'item_statistical_code_ids',
 'item_statistical_codes',
 'item_status',
 'item_temporary_location_id',
 'item_temporary_location_name',
 'item_updated_date']

In [3]:
raw_call_numbers = raw_df["item_effective_call_number"]
expected_call_numbers = expected_df["item_effective_call_number"]

In [4]:
sorted_call_numbers = raw_call_numbers.sort_values().reset_index(drop=True)
expected_call_numbers_copy = expected_call_numbers.reset_index(drop=True)

simple_sort = pd.DataFrame({
    "sorted_call_numbers": sorted_call_numbers,
    "expected_call_numbers": expected_call_numbers_copy
})


The raw call numbers need cleaning. When comparing values, some have missing decimal places, like this:

my_sort

| Index | Raw Call Number         | Expected Call Number    |
|-------|-------------------------|-------------------------|
| 0     | 137 529 THS, 1          | 137.529 THS, 1         |
| 1     | 138 093 THS, 1          | 138.093 THS, 1         |

and some values are sorted incorrectly because the sort is done as a string comparison instead of the LoC way

| Index | Raw Call Number              | Expected Call Number        |
|-------|------------------------------|-----------------------------|
| 9     | B105.A8 E24 2004 VideoDVD, 1 | B72.G73 A7 2008 VideoDVD, 1 |
| 10    | B105.W6 F465 2009 VideoDVD, 1 | B105.A8 E24 2004 VideoDVD, 1 |
| 11    | B105.W6 F465 2009 VideoDVD, 1 | B105.W6 F465 2009 VideoDVD, 1 |
| 12    | B105.W6 F465 2009 VideoDVD, 1 | B105.W6 F465 2009 VideoDVD, 1 |
| 13    | B105.W6 F465 2009 VideoDVD, 1 | B105.W6 F465 2009 VideoDVD, 1 |
| 14    | B105.W6 F465 2009 VideoDVD, 1 | B105.W6 F465 2009 VideoDVD, 1 |
| 15    | B105.W6 F465 2009 VideoDVD, 1 | B105.W6 F465 2009 VideoDVD, 1 |
| 16    | B108 .R66 2002 VideoDVD, 1   | B105.W6 F465 2009 VideoDVD, 1 |
| 17    | B108 .R66 2002 VideoDVD, 1   | B108.R66 2002 VideoDVD, 1   |



In [5]:
def normalize_call_number(call_number):
    # Remove space between opening letters and numbers (e.g., "PR 1109" -> "PR1109")
    call_number = re.sub(r'([A-Z]+) (\d+)', r'\1\2', call_number)

    # Remove space before the first period (e.g., "PR1109 .A2" -> "PR1109.A2")
    call_number = re.sub(r'(\d) \.', r'\1.', call_number)

    return call_number

raw_call_number = "PN1995.9.C47 T8 2005 VideoDVD, 1"
fixed_call_number = normalize_call_number(raw_call_number)

fixed_call_number

'PN1995.9.C47 T8 2005 VideoDVD, 1'

In [6]:
normalized_raw_callnumbers = raw_call_numbers.apply(normalize_call_number)

raw_vs_normalized = pd.DataFrame({
    "raw": raw_call_numbers,
    "normalized": normalized_raw_callnumbers
})

raw_vs_normalized_different = raw_vs_normalized[raw_vs_normalized["raw"] != raw_vs_normalized["normalized"]]

raw_vs_normalized_different.reset_index(drop=True, inplace=True)

`raw_vs_normalized_different`, shows all rows that have been been changed after normalization. Now that the rows have been normalized, we need a comparator made for the LoC classification.

In [36]:
def loc_sort_key(call_number):
    """
    Generate a key for sorting LoC call numbers.
    - Ensures consistent structure by padding missing sections.
    - Splits letters, numbers, and decimals correctly.
    """
    parts = re.findall(r'[A-Za-z]+|\d+\.\d+|\d+', call_number)

    # Convert numeric parts to float, keep letters as-is
    processed_parts = [(part if part.isalpha() else float(part)) for part in parts]
    processed_parts = [part for part in parts]

    return processed_parts


print(raw_call_number, loc_sort_key(raw_call_number))

print("HV6049.O437 2005 VideoDVD, 1", loc_sort_key("HV6049.O437 2005 VideoDVD, 1"))
print("PS3563.C3868 H68 1998 VideoDVD, 1", loc_sort_key("PS3563.C3868 H68 1998 VideoDVD, 1"))

PN1995.9.C47 T8 2005 VideoDVD, 1 ['PN', '1995.9', 'C', '47', 'T', '8', '2005', 'VideoDVD', '1']
HV6049.O437 2005 VideoDVD, 1 ['HV', '6049', 'O', '437', '2005', 'VideoDVD', '1']
PS3563.C3868 H68 1998 VideoDVD, 1 ['PS', '3563', 'C', '3868', 'H', '68', '1998', 'VideoDVD', '1']


In [38]:
raw_vs_normalized["loc_sort_key"] = raw_vs_normalized["normalized"].apply(loc_sort_key)
raw_vs_normalized["sort_key_len"] = raw_vs_normalized["loc_sort_key"].apply(len)

raw_vs_normalized.groupby("sort_key_len", group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), 5), random_state=42)
)


normalized_sort = raw_vs_normalized.sort_values(by="loc_sort_key", key=lambda x: x.apply(tuple))

/var/folders/r7/zs0r39wj3jjgsrj0zy75r9d40000gn/T/ipykernel_53041/2228980848.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  raw_vs_normalized.groupby("sort_key_len", group_keys=False).apply(
